In [1]:
import torch
import torch.nn as nn
from torch import optim as s
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools
import bz2

In [2]:
device=torch.device("cpu")

In [3]:
lines_filepath=os.path.join("cornell movie-dialogs corpus","movie_lines.txt")
conv_filepath=os.path.join("cornell movie-dialogs corpus","movie_conversations.txt")

In [4]:
with open(lines_filepath,"r") as file:
    lines=file.readlines()
#for line in lines[:8]:
    #print(line.strip())

In [5]:
line_fields=["lineID","characterID","movieID","character","text"]
lines={}
with open(lines_filepath,"r",encoding="iso-8859-1") as f:
    for line in f:
        values=line.split(" +++$+++ ")
        lineobj={}
        for i,field in enumerate(line_fields):
            lineobj[field]=values[i]
        lines[lineobj["lineID"]]=lineobj    

In [6]:
#lines

In [7]:
with open(conv_filepath,'r',encoding="iso-8859-1") as f:
    s_lines=f.readlines()
#s_lines

In [8]:
conv_fields=["characterID","character2ID","movieID","utteranceIDs"]
conversations=[]
with open(conv_filepath,'r',encoding="iso-8859-1") as f:
    for line in f:
        values=line.split(" +++$+++ ")
        convobj={}
        for i,field in enumerate(conv_fields):
            convobj[field]=values[i]
        lineIds=eval(convobj["utteranceIDs"])
        convobj["lines"]=[]
        for lineId in lineIds:
            convobj["lines"].append(lines[lineId])
        conversations.append(convobj)    

In [9]:
#conversations

In [10]:
qa_pairs=[]
for conversation in conversations:
    for i in range(len(conversation["lines"])-1):
        inputLine=conversation["lines"][i]["text"].strip()
        targetLine=conversation["lines"][i+1]["text"].strip()
        if inputLine and targetLine:
            qa_pairs.append([inputLine,targetLine])

In [11]:
#qa_pairs

In [12]:
datafile=os.path.join("cornell movie-dialogs corpus","fomatted_movie_lines.txt")
delimiter='\t'
delimiter=str(codecs.decode(delimiter,"unicode_escape"))
#print("\nwriting newly formatted file")
with open(datafile,'w',encoding="utf-8") as outputfile:
    writer=csv.writer(outputfile,delimiter=delimiter)
    for pair in qa_pairs:
        writer.writerow(pair)
#print("done writing to file")

In [13]:
datafile=os.path.join("cornell movie-dialogs corpus","fomatted_movie_lines.txt")
with open(datafile,'rb') as file:
    lines=file.readlines()
#for line in lines[:8]:
    #print(line)

In [14]:
PAD_token=0
SOS_token=1
EOS_token=2

class vocabulary:
    
    def __init__(self,name):
        self.name=name
        self.word2index={}
        self.word2count={}
        self.index2word={PAD_token:"PAD",SOS_token:"SOS",EOS_token:"EOS"}
        self.num_words=3
    
    def addsentence(self,sentence):
        for word in sentence.split(' '):
            self.addword(word)
            
    def addword(self,word):
        if word not in self.word2index:
            self.word2index[word]=self.num_words
            self.word2count[word]=1
            self.index2word[self.num_words]=word
            self.num_words+=1
        else:
            self.word2count[word]+=1
            
    def trim(self,min_word):
        keep_words=[]
        for k,v in self.word2count.items():
            if v>=min_word:
                keep_words.append(k)
        #print("keep_words {} / {}={:.4f}".format((len(keep_words)),self.num_words,len(keep_words)/self.num_words))
        self.word2index={}
        self.word2count={}
        self.index2word={PAD_token:"PAD",SOS_token:"SOS",EOS_token:"EOS"}
        self.num_words=3
        for word in keep_words:
            self.addword(word)
            
    
            
            

In [15]:
def unicodeToAscii(s):
    return ''.join(c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c)!='Mn')

In [16]:
#unicodeToAscii("Maître Renard, par l’odeur alléché")

In [17]:
def normalisestring(s):
    s=unicodeToAscii(s.lower().strip())
    s=re.sub(r"([.!?])",r" \1",s)
    s=re.sub(r"([^a-zA-Z.!?]+)",r" ",s)
    s=re.sub(r"\s+",r" ",s).strip()
    
    return s
    

In [18]:
datafile=os.path.join("cornell movie-dialogs corpus","fomatted_movie_lines.txt")
delimiter='\t'
delimiter=str(codecs.decode(delimiter,"unicode_escape"))
#print("\nwriting newly formatted file")
with open(datafile,'w',encoding="utf-8") as outputfile:
    writer=csv.writer(outputfile,delimiter=delimiter)
    for pair in qa_pairs:
        pair[0]=normalisestring(pair[0])
        pair[1]=normalisestring(pair[1])
        writer.writerow(pair)
#print("done writing to file")

In [19]:
datafile=os.path.join("cornell movie-dialogs corpus","fomatted_movie_lines.txt")
#print("Reading and processing file.....please wait")
lines=open(datafile,encoding="utf-8").read().strip().split('\n')
pairs=[[normalisestring(s) for s in pair.split('\t')] for pair in lines]
#print("Done reading")
voc=vocabulary("cornell movie-dialogs corpus")

In [20]:
MAX_LENGTH=10
def filterpair(p):
    return (len(p[0].split())<MAX_LENGTH and len(p[1].split())<MAX_LENGTH)
def filterpairs(pairs):
    return [pair for pair in pairs if filterpair(pair)] 

In [21]:
pairs=[pair for pair in pairs if len(pair)>1]
#print("There are {} pairs/conversations in the dataset".format(len(pairs)))
pairs=filterpairs(pairs)
#print("After filtering there are {} pairs/conversations in the dataset".format(len(pairs)))

In [22]:
for pair in pairs:
    voc.addsentence(pair[0])
    voc.addsentence(pair[1])
print("counted words:",voc.num_words)
#for pair in pairs[:10]:
#   print(pair)

counted words: 17993


In [23]:
MIN_COUNT=3

def trimrarewords(voc,pairs,MIN_COUNT):
    voc.trim(MIN_COUNT)
    keep_pairs=[]
    for pair in pairs:
        input_sentence=pair[0]
        output_sentence=pair[1]
        keep_input=True
        keep_output=True
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input=False
                break
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output=False
                break
        if keep_input and keep_output:
            keep_pairs.append(pair)
    #print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs),len(keep_pairs),len(keep_pairs)/len(pairs)))
    return keep_pairs

In [24]:
pairs=trimrarewords(voc,pairs,MIN_COUNT)

In [25]:
#pairs[0][1]

In [26]:
def indexesfromsentence(voc,sentence):
    return [voc.word2index[word] for word in sentence.split(' ')]+[EOS_token]

In [27]:
#for pair in pairs:
#pairs[0][0]
#print(indexesfromsentence(voc,pairs[3][0]))

In [28]:
#pairs[4][0]

In [29]:
#pairs[1][0]

In [30]:
a=[[1,2,3],[2,1]]
a.append([1])
#print(a)

In [31]:
inp=[]
out=[]
for pair in pairs[:10]:
    inp.append(pair[0])
    out.append(pair[1])
#print(len(inp))
indexes=[indexesfromsentence(voc,sentence) for sentence in inp]
#indexes

In [32]:
a=[1,2,3,4,5]
b=['a','s','d']
#print(list(zip(a,b)))
#list(itertools.zip_longest(a,b))

In [33]:
#list(itertools.zip_longest(*indexes,fillvalue=0))

In [34]:
def zeropadding(l,fillvalue=0):
    return list(itertools.zip_longest(*l,fillvalue=fillvalue))

In [35]:
leng=[len(index) for index in indexes]
#max(leng)

In [36]:
test_result=zeropadding(indexes)
#print(len(test_result))
#test_result

In [37]:
def binaryMatrix(l,value=0):
    m=[]
    for i,seq in enumerate(l):
        m.append([])
        for token in seq:
            if token==PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

In [38]:
a=[[1,2,3],[2,3]]
a[1].append(2)
#a

In [39]:
def inputVar(l,voc):
    indexes_batch=[indexesfromsentence(voc,sentence) for sentence in l]
    lengths=[len(indexes) for indexes in indexes_batch]
    padList=zeropadding(indexes_batch)
    padVar=torch.LongTensor(padList)
    
    return padVar,lengths

In [40]:
def outputVar(l,voc):
    indexes_batch=[indexesfromsentence(voc,sentence) for sentence in l]
    max_target_length=max([len(indexes) for indexes in indexes_batch])
    padList=zeropadding(indexes_batch)
    mask=binaryMatrix(padList)
    mask=torch.ByteTensor(mask)
    padVar=torch.LongTensor(padList)
    
    return padVar,mask,max_target_length

In [41]:
def batchTraindata(voc,pair_batch):
    pair_batch.sort(key=lambda x:len(x[0].split(' ')),reverse=True)
    input_batch,output_batch=[],[]
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp,lengths=inputVar(input_batch,voc)
    output,mask,max_target_length=outputVar(output_batch,voc)
    
    return inp,lengths,output,mask,max_target_length

In [42]:
small_batch_size=5
batches=batchTraindata(voc,[random.choice(pairs) for _ in range(small_batch_size)])
input_variable,lengths,target_variable,mask,max_target_length=batches
#print("input_variable:")
p=torch.nn.utils.rnn.pack_padded_sequence(input_variable,lengths)
#print(p)
#print("lengths:")
#print(lengths)
#print("target_variable:")
#print(target_variable)
#print("mask:")
#print(mask)
#print("max_target_length:")
#print(max_target_length)
#print(torch.nn.utils.rnn.pad_packed_sequence(p))
#print(target_variable[max_target_length-1])

In [43]:
class EncoderRNN(nn.Module):
    def __init__(self,hidden_size,embedding,n_layers=1,dropout=0):
        super(EncoderRNN,self).__init__()
        self.n_layers=n_layers
        self.hidden_size=hidden_size
        self.embedding=embedding
        self.gru=nn.GRU(hidden_size,hidden_size,n_layers,dropout=(0 if n_layers==1 else dropout),bidirectional=True)
    def forward(self,input_seq,input_lengths,hidden=None):
        embedded=self.embedding(input_seq)
        packed=torch.nn.utils.rnn.pack_padded_sequence(embedded,input_lengths)
        outputs,hidden=self.gru(packed,hidden)
        outputs,_=torch.nn.utils.rnn.pad_packed_sequence(outputs)
        outputs=outputs[:,:,:self.hidden_size]+outputs[:,:,self.hidden_size:]
        
        return outputs,hidden
        

In [44]:
class Attn(torch.nn.Module):
    def __init__(self,method,hidden_size):
        super(Attn,self).__init__()
        self.method=method
        self.hidden_size=hidden_size
    def dot_score(self,hidden,encoder_outputs):
        return torch.sum(hidden*encoder_outputs,dim=2)
    def forward(self,hidden,encoder_outputs):
        attn_energies=self.dot_score(hidden,encoder_outputs)
        attn_energies=attn_energies.t()
        return F.softmax(attn_energies,dim=1).unsqueeze(1)

In [45]:
class LongAttnDecoderRNN(nn.Module):
    def __init__(self,attn_model,embedding,hidden_size,output_size,n_layers=1,dropout=0.1):
        super(LongAttnDecoderRNN,self).__init__()
        self.attn_model=attn_model
        self.hidden_size=hidden_size
        self.output_size=output_size
        self.n_layers=n_layers
        self.dropout=dropout
        
        self.embedding=embedding
        self.embedding_dropout=nn.Dropout(dropout)
        self.gru=nn.GRU(hidden_size,hidden_size,n_layers,dropout=(0 if n_layers==1 else dropout))
        self.concat=nn.Linear(2*hidden_size,hidden_size)
        self.out=nn.Linear(hidden_size,output_size)
        
        self.attn=Attn(attn_model,hidden_size)
    def forward(self,input_step,last_hidden,encoder_outputs):
        embedded=self.embedding(input_step)
        embedded=self.embedding_dropout(embedded)
        rnn_output,hidden=self.gru(embedded,last_hidden)
        attn_weights=self.attn(rnn_output,encoder_outputs)
        context=attn_weights.bmm(encoder_outputs.transpose(0,1))
        rnn_output=rnn_output.squeeze(0)
        context=context.squeeze(1)
        concat_input=torch.cat((rnn_output,context),1)
        concat_output=torch.tanh(self.concat(concat_input))
        output=self.out(concat_output)
        output=F.softmax(output,dim=1)
        return output,hidden
        

In [46]:
def maskNLLLoss(decoder_output,target,mask):
    nTotal=mask.sum()
    target=target.view(-1,1)
   
    gathered_tensor=torch.gather(decoder_output,1,target)
    crossEntropy=-torch.log(gathered_tensor)
    loss=crossEntropy.masked_select(mask)
    loss=torch.mean(loss)
    loss=loss.to(device)
    return loss,nTotal.item()

In [47]:
small_batch_size=5
batches=batchTraindata(voc,[random.choice(pairs) for _ in range(small_batch_size)])
input_variable,lengths,target_variable,mask,max_target_length=batches
#print("input_variable shape:")
#print(input_variable.shape)
lengths=torch.tensor(lengths)
#print("lengths shape:")
#print(lengths.shape)
#print("target_variable shape:")
#print(target_variable.shape)
#print("mask shape:")
#print(mask.shape)
#print("max_target_length:")
#print(max_target_length)
hidden_size=500
encoder_n_layers=2
decoder_n_layers=2
dropout=0.1
attn_model='dot'
embedding=nn.Embedding(voc.num_words,hidden_size)
encoder=EncoderRNN(hidden_size,embedding,encoder_n_layers,dropout)
decoder=LongAttnDecoderRNN(attn_model,embedding,hidden_size,voc.num_words,decoder_n_layers,dropout)
encoder=encoder.to(device)
decoder=decoder.to(device)
encoder.train()
decoder.train()
encoder_optimizer=optim.Adam(encoder.parameters(),lr=0.0001)
decoder_optimizer=optim.Adam(decoder.parameters(),lr=0.0001)
input_variable=input_variable.to(device)
lengths=lengths.to(device)
target_variable=target_variable.to(device)
mask=mask.to(device)
loss=0
print_losses=[]
n_totals=0

encoder_outputs,encoder_hidden=encoder(input_variable,lengths)
#print("Encoder output's shape:",encoder_outputs.shape)
#print("Last encoder hidden shape:",encoder_hidden.shape)
decoder_input=torch.LongTensor([[SOS_token for _ in range(small_batch_size)]])
decoder_input=decoder_input.to(device)
#print("Initial decoder input shape",decoder_input.shape)
#print(decoder_input)
decoder_hidden=encoder_hidden[:decoder.n_layers]
#print("Initial decoder hidden state shape",decoder_hidden.shape,"\n")
#print("----------------------------------------------------------------")
#print("Now let's look what's happening in every timestep of the GRU")
#print("----------------------------------------------------------------")
#print("\n")

for t in range(max_target_length):
    decoder_output,decoder_hidden=decoder(decoder_input,decoder_hidden,encoder_outputs)
    #print("Decoder output shape:",decoder_output.shape)
    #print("Decoder hidden shape:",decoder_hidden.shape)
    
    decoder_input=target_variable[t].view(1,-1)
    #print("The target variable at the current timestep before reshaping",target_variable[t])
    #print("The target variable at the current timestep shape after reshaping",target_variable[t].shape)
    #print("The decoder input shape (reshape the target variable)",decoder_input.shape)
    
    #print("The mask at the current timestep",mask[t])
    #print("The mask at the current timestep shape",mask[t].shape)
    mask_loss,nTotal=maskNLLLoss(decoder_output,target_variable[t],mask[t])
    #print("Mask loss:",mask_loss)
    #print("Total:",nTotal)
    loss+=mask_loss
    print_losses.append(mask_loss.item()*nTotal)
    n_totals+=nTotal
    #print(n_totals)
    encoder_optimizer.step()
    encoder_optimizer.step()
    returned_loss=sum(print_losses)/n_totals
    #print("Returned_loss:",returned_loss)
    #print("\n")
    #print('-------------------------------DONE ONE TIMESTEP--------------------------------')
    #print("\n")

In [48]:
def train(input_variable,lengths,target_variable,mask,max_target_len,encoder,decoder,embedding,encoder_optimizer,
          decoder_optimizer,batch_size,clip,max_length=MAX_LENGTH):
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_variable=input_variable.to(device)
    lengths=torch.tensor(lengths).to(device)
    target_variable=target_variable.to(device)
    mask=mask.to(device)
      
    loss=0
    print_losses=[]
    n_totals=0
    
    encoder_outputs,encoder_hidden=encoder(input_variable,lengths)
    
    decoder_input=torch.tensor([[SOS_token for _ in range(batch_size)]])
    decoder_input=decoder_input.to(device)
    
    decoder_hidden=encoder_hidden[:decoder.n_layers]
    
    use_teacher_forcing=True if random.random() > teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output,decoder_hidden=decoder(decoder_input,decoder_hidden,encoder_outputs)
            
            decoder_input=target_variable[t].view(1,-1)
            
            mask_loss,nTotal=maskNLLLoss(decoder_output,target_variable[t],mask[t])
            loss+=mask_loss
            print_losses.append(mask_loss.item()*nTotal)
            n_totals+=nTotal
           
            
    else:
        for t in range(max_target_len):
            decoder_output,decoder_hidden=decoder(decoder_input,decoder_hidden,encoder_outputs)
            _,topi=decoder_output.topk(1)
            decoder_input=torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            
            mask_loss,nTotal=maskNLLLoss(decoder_output,target_variable[t],mask[t])
            loss+=mask_loss
            print_losses.append(mask_loss.item()*nTotal)
            n_totals+=nTotal
    
    loss.backward()
    
    torch.nn.utils.clip_grad_norm_(encoder.parameters(),clip)
    torch.nn.utils.clip_grad_norm_(decoder.parameters(),clip)

    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return sum(print_losses)/n_totals

In [49]:
def trainIters(model_name,voc,pairs,encoder,decoder,encoder_optimizer,decoder_optimizer,embedding,encoder_n_layers,
               decoder_n_layers,save_dir,n_iteration,branch_size,print_every,save_every,clip,corpus_name,loadFilename):
    training_batches=[batchTraindata(voc,[random.choice(pairs) for _ in range(batch_size)]) for _ in range(n_iteration)]
    #print('Initialising.........')
    start_iteration=1
    print_loss=0
    if loadFilename:
        start_iteration=checkpoint['iteration']+1
    #print('training....')
    for iteration in range(start_iteration,n_iteration+1):
        training_batch=training_batches[iteration-1]
        input_variable,lengths,target_variable,mask,max_target_len=training_batch
        
        loss=train(input_variable,lengths,target_variable,mask,max_target_len,encoder,decoder,embedding,encoder_optimizer,decoder_optimizer,
                  batch_size,clip)
        print_loss+=loss
        if iteration % print_every == 0:
            print_avg_loss=print_loss/print_every
            #print("Iteration:{},percent complete:{},Average loss:{}".format(iteration,iteration*100/n_iteration,print_avg_loss))
            print_loss=0
        if iteration % save_every == 0:
            directory=os.path.join(save_dir,model_name,corpus_name,'{}-{}_{}'.format(encoder_n_layers,decoder_n_layers,hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({'iteration':iteration,'en':encoder.state_dict(),'de':decoder.state_dict(),'en_opt':encoder_optimizer.state_dict(),
                        'de_opt':decoder_optimizer.state_dict(),'loss':loss,'voc_dict':voc.__dict__,'embedding':embedding.state_dict()},
                       os.path.join(directory,'{}_{}.tar'.format(iteration,'checkpoint')))

In [50]:
class GreedySearchDecoder(nn.Module):
    def __init__(self,encoder,decoder):
        super(GreedySearchDecoder,self).__init__()
        self.encoder=encoder
        self.decoder=decoder
    def forward(self,input_seq,input_length,max_length):
        encoder_outputs,encoder_hidden=self.encoder(input_seq,input_length)
        decoder_hidden=encoder_hidden[:decoder.n_layers]
        decoder_input=torch.ones(1,1,device=device,dtype=torch.long)*SOS_token
        all_tokens=torch.zeros([0],device=device,dtype=torch.long)
        all_scores=torch.zeros([0],device=device,dtype=torch.long)
        for _ in range(max_length):
            decoder_output,decoder_hidden=self.decoder(decoder_input,decoder_hidden,encoder_outputs)
            decoder_score,decoder_input=torch.max(decoder_output,dim=1)
            all_scores=torch.cat((all_scores,torch.LongTensor([decoder_score])),dim=0)
            all_tokens=torch.cat((all_tokens,torch.LongTensor([decoder_input])),dim=0)
            decoder_input=torch.unsqueeze(decoder_input,0)
            
        return all_tokens,all_scores

In [51]:
def evaluate(encoder,decoder,searcher,voc,sentence,max_length=MAX_LENGTH):
    indexes_batch=[indexesfromsentence(voc,sentence)]
    lengths=torch.tensor([len(indexes) for indexes in indexes_batch])
    input_batch=torch.LongTensor(indexes_batch).transpose(0,1)
    input_batch=input_batch.to(device)
    lengths=lengths.to(device)
    tokens,scores=searcher(input_batch,lengths,max_length)
    decoded_words=[voc.index2word[token.item()] for token in tokens]
    return decoded_words

def evaluateInput(encoder,decoder,searcher,voc):
    input_sentence=''
    p=[]
    n_iteration=400
    i=0
    while(1):
        try:
            input_sentence=input('> ')
            if (input_sentence=='q' or input_sentence=='quit'):
                break
            input_sentence=normalisestring(input_sentence)
            output_words=evaluate(encoder,decoder,searcher,voc,input_sentence)
            output_words[:]=[x for x in output_words if not(x=='EOS' or x=='PAD')]
            print('BOT:',' '.join(output_words))
            s=input('> ')
            if s!='y' and s!='q':
                p.append([])
                s=normalisestring(s)
                p[i].append(input_sentence)
                p[i].append(s)
                n_iteration+=1
                i+=1
        
        except KeyError:
            print("Error:Encountered unknown word.")    
    
    if p!=[] and input_sentence=='q':
        print_every=1
        save_every=1
                
        datafile=os.path.join("cornell movie-dialogs corpus","fomatted_movie_lines.txt")
        delimiter='\t'
        delimiter=str(codecs.decode(delimiter,"unicode_escape"))
        print("\nwriting newly formatted file")
        with open(datafile,'a',encoding="utf-8") as outputfile:
            writer=csv.writer(outputfile,delimiter=delimiter)        
            for pair in p:
                if pair not in pairs:
                    print(1)
                    writer.writerow(pair)
        
        

In [52]:
model_name='cb_model'
attn_model='dot'
hidden_size=500
encoder_n_layers=2
decoder_n_layers=2
dropout=0.2
batch_size=100

clip=50

teacher_forcing_ratio=1.0
learning_rate=0.00005
decoder_learning_ratio=5.0

#loadFilename=os.path.join("cornell movie-dialogs corpus","cb_model","trained_data","2-2_500","70000_checkpoint.tar")
checkpoint_iter=4000
loadFilename=None
if loadFilename:
    checkpoint=torch.load(loadFilename)
    encoder_sd=checkpoint['en']
    decoder_sd=checkpoint['de']
    encoder_optimizer_sd=checkpoint['en_opt']
    decoder_optimizer_sd=checkpoint['de_opt']
    embedding_sd=checkpoint['embedding']
    voc.__dicy__=checkpoint['voc_dict']
#print("building and encoder.....")
embedding=nn.Embedding(voc.num_words,hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
    encoder=EncoderRNN(hidden_size,embedding,encoder_n_layers,dropout)
    decoder=LongAttnDecoderRNN(attn_model,embedding,hidden_size,voc.num_words,decoder_n_layers,dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
encoder=encoder.to(device)
decoder=decoder.to(device)
#print('models built and ready to go')    

In [53]:
clip=50
teacher_forcing_ratio=1.0
learning_rate=0.000001
decoder_learning_ratio=5.0
n_iteration=70500
print_every=1
save_every=500

encoder.train()
decoder.train()

#print("building optimizers")
encoder_optimizer=optim.Adam(encoder.parameters(),lr=learning_rate)
decoder_optimizer=optim.Adam(decoder.parameters(),lr=learning_rate*decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)
#print("start training!")
trainIters(model_name,voc,pairs,encoder,decoder,encoder_optimizer,decoder_optimizer,embedding,encoder_n_layers,decoder_n_layers,
           "cornell movie-dialogs corpus",n_iteration,batch_size,print_every,save_every,clip,"trained_data",loadFilename)

In [54]:
encoder.eval()

decoder.eval()
searcher=GreedySearchDecoder(encoder,decoder)
evaluateInput(encoder,decoder,searcher,voc)

In [55]:
pair=["hey",normalisestring("hey,how are you?")]
c=1 if pair in pairs else 0
c

In [56]:
p

In [57]:
d=os.path.join("ubuntu_dialogs.tgz")
with open(d,"r",encoding="iso-8859-1") as f:
    lines=f.readlines()
    

In [58]:
p=[['aadfd'],['adadg']]
p[0].append('fa')
p

In [59]:
p=[]
#p.append(['gda'])
print(p)

In [60]:
a=1 if "word" in voc.index2word.values() else 0
a

In [61]:
voc.word2index['bot']

In [62]:
voc.num_words


In [63]:
e=encoder.embedding
e(torch.tensor([7826]))

In [64]:
for i in range(29500,68000,500):
    filename=str(i)+("_checkpoint.tar")
    loadFilename=os.path.join("cornell movie-dialogs corpus","cb_model","trained_data","2-2_500",filename)

    if loadFilename:
        checkpoint=torch.load(loadFilename)
        loss=checkpoint['loss']
        print(i,":",loss)

In [65]:
for i in range(0,10,2):
    print(str(i)+('weeee'))